In [1]:
import os
import streamlit as st
from langchain_groq import ChatGroq
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

from langchain.chains import create_history_aware_retriever
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.retrieval import create_retrieval_chain


from langchain_core.chat_history  import BaseChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder

# -----------------------------------------------------------------

In [ ]:
#History Aware Chains
        contextualize_prompt=(
                                        "Given a chat history and the latest user question"
                                        "which might reference context in the chat history,"
                                        "formulate a standalone question which can be understood"
                                        "without the chat history. Do NOT answer the question,"
                                        "just reformulate if it needed and otherwise return it as is"    
                                    )
        prompt=ChatPromptTemplate.from_messages(
            [
                ('system',contextualize_prompt),
                MessagesPlaceholder('chat_history'),
                ('human','{input}'),
            ]
        )
        
        history_aware_retriever=create_history_aware_retriever(llm,retriever,prompt)

## Rephrasing the Question:
The contextualize_prompt is used to instruct the model to take the chat history and the latest user question, and then reformulate the question if necessary. This ensures that the question can be understood without needing the chat history.
The ChatPromptTemplate.from_messages creates a template that includes the system’s instructions (contextualize_prompt), the chat history (MessagesPlaceholder('chat_history')), and the user’s input ('human', '{input}').
## Fetching Documents:
The create_history_aware_retriever(llm, retriever, prompt) function uses the prompt to generate a search query that incorporates the context from the chat history.
This search query is then passed to the retriever, which fetches relevant documents based on the query.
Summary
## Rephrasing: 
        The model reformulates the user’s question using the chat history to make it standalone.
## Retrieving:
         The reformulated question is used to fetch relevant documents, ensuring the retrieval process is context-aware.

## How It Works
Rephrasing the Question: The history_aware_retriever uses the prompt to reformulate the user’s question based on the chat history.

Generating a Search Query: This reformulated question is used to create a search query.

Fetching Documents: The search query is passed to the retriever, which fetches the relevant documents.

# -----------------------------------------------------------------------

In [ ]:
#Question Answering chain
        system_prompt = (
            "You are an assistant for question-answering tasks. "
            "Use the following pieces of retrieved context to answer "
            "the question. If you don't know the answer, say that you"
            "don't know. Use three sentences maximum and keep the"
            "answer concise."
            "\n\n"
            "{context}") 
        
        qa_prompt=ChatPromptTemplate.from_messages(
            [
                ('system',system_prompt),
                MessagesPlaceholder('chat_history'),
                ('human','{input}'),
            ]
        )
        question_answer_chain=create_stuff_documents_chain(llm,qa_prompt)
        rag_chain=create_retrieval_chain(history_aware_retriever,question_answer_chain)
        
        conversational_rag_chain=RunnableWithMessageHistory(
                                                                rag_chain,
                                                                get_session_history,
                                                                input_messages_key='input',
                                                                history_messages_key='chat_message',
                                                                output_messages_key='answer'

## Question Answering Chain:
Purpose: To answer the user’s question using retrieved context.

Prompt: system_prompt provides instructions for the model to answer the question concisely using the retrieved context. If the model doesn’t know the answer, it should say so.

Template: Another ChatPromptTemplate.from_messages is used here to structure the conversation, including the system’s instructions, the chat history, and the user’s input. This ensures the model can provide a relevant and concise answer based on the context.